In [34]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

ds = load_dataset("rajpurkar/squad")

model = AutoModelForCausalLM.from_pretrained("./cqatrains1")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

model.eval()
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [72]:
data="Notre Dame is known for its competitive admissions, with the incoming class enrolling in fall 2015 admitting 3,577 from a pool of 18,156 (19.7%). The academic profile of the enrolled class continues to rate among the top 10 to 15 in the nation for national research universities. The university practices a non-restrictive early action policy that allows admitted students to consider admission to Notre Dame as well as any other colleges to which they were accepted. 1,400 of the 3,577 (39.1%) were admitted under the early action plan. Admitted students came from 1,311 high schools and the average student traveled more than 750 miles to Notre Dame, making it arguably the most representative university in the United States. While all entering students begin in the College of the First Year of Studies, 25% have indicated they plan to study in the liberal arts or social sciences, 24% in engineering, 24% in business, 24% in science, and 3% in architecture.What percentage of students were admitted to Notre Dame in fall 2015?"
tokenized=tokenizer(data, padding='longest', max_length=512, truncation=True, return_tensors="pt")
tokenized

{'input_ids': tensor([[128000,   2688,    265,  41798,    374,   3967,    369,   1202,  15022,
          48911,     11,    449,    279,  19957,    538,    665,  16608,    304,
           4498,    220,    679,     20,  55689,    220,     18,     11,  23411,
            505,    264,   7463,    315,    220,    972,     11,  10132,    320,
            777,     13,     22,  53172,    578,  14584,   5643,    315,    279,
          37191,    538,   9731,    311,   4478,   4315,    279,   1948,    220,
            605,    311,    220,    868,    304,    279,   7140,    369,   5426,
           3495,  23978,     13,    578,  12374,  12659,    264,   2536,   5621,
           6765,    535,   4216,   1957,   4947,    430,   6276,  16584,   4236,
            311,   2980,  26360,    311,  44564,  41798,    439,   1664,    439,
            904,   1023,  31252,    311,    902,    814,   1051,  11928,     13,
            220,     16,     11,   3443,    315,    279,    220,     18,     11,
          2341

In [73]:
output = model(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask'], labels=tokenized['input_ids'])

In [74]:
logits=output.logits

In [76]:
import torch
import torch.nn.functional as F

probabilities = F.softmax(logits, dim=-1)
probabilities = probabilities[0][-1]
probabilities.shape

topk_values, top3_indices = torch.topk(probabilities, k=3)

prob_top=tokenizer.decode(top3_indices)
print(prob_top)

6324
